## 모델링

In [ ]:
#한글 깨짐 -> 설치 후 런타임 다시 시작 필요!
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 4 dirs
/usr/share/fonts/truetype/dejavu: caching, new cache contents: 22 fonts, 0 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconf

In [ ]:
# MeCab, nltk 설치
!pip install nltk
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-08-27 11:31:32--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-08-27 11:31:32--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6c99f478b813a44949304c269f.dl.dropboxusercontent.com/cd/0/get/BrzKNy2q2VG5HVu7-6KxSuKSmRV27iZe1F9hmGMjJYh-CCcFXtmai-EsYhdzhp88TzpU0Dwy9xbdG-DlbWO9XGMNY_i0eIAYVxHmmQODdJV8EHZqpQPkoq84PhhAU-l5Qy7wd3_f4Ouiw4GlJMngfL1cdnILTT72t1DCTTnVgdu1j1zG7XjUk0jR8EVO-EFu_xE/file?dl=1# [following

In [66]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import random as rn
import re
import os
import warnings
import folium
import geopy.distance
import konlpy
import MeCab
from collections import Counter
from konlpy.tag import Kkma, Komoran, Hannanum, Okt, Mecab
from konlpy.utils import pprint
from nltk import Text
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.layers import Dense, Activation
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


plt.rc('font',  family='NanumBarunGothic')
plt.rcParams["axes.unicode_minus"]=False #마이너스 깨짐 해결
%matplotlib inline
warnings.filterwarnings(action='ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (1) 데이터 불러오기

In [33]:
data="/content/drive/MyDrive/Project/팀 프로젝트/3_company dinner/data/"

In [34]:
df = pd.read_csv(os.path.join(data,"compnay_dinner_tokenization_2019.csv"),index_col=0)
df['review']=df['review'].fillna("")
df['review_check']=df['review_check'].fillna("")
df.head()

,store_name,date,review,length,review_check,check_length,menu,mood,service,price,...,기리야마,시카고,대려도,신부자,봉추찜닭,리샨,Pavox,삼거리,느린마을,봉피양
0,백억하누 강남본점,2020-01-03,네이버로 예약하고 방문했는데 사정이 있어 조용한 구석자리를 부탁한다고 썼는데도 구석...,358,네이버로 예약하고 방문했는데 사정이 있어 조용한 구석 자리를 부탁한다고 썼는데도 구...,370,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,강남진해장,2022-04-05,일반적인 양평해장국과 다르게 기본 간이 되어있고 베이스가 육개장과 뼈해장국같은 느...,343,일반적인 양평 해장국과 다르게 기본 간이 되어 있고 베이스가 육개장과 뼈해장국 같은...,348,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,백억하누 강남본점,2022-05-07,개인룸이 있고 룸안에 화장실이 있어 너무 편하고 좋았는데 옆에 개인 서랍이 있는지...,163,개인 룸이 있고 룸 안에 화장실이 있어 너무 편하고 좋았는데 옆에 개인 서랍이 있는...,165,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,딘타이펑 강남점,2019-11-21,분위기도 괜찮고 맛있어요 네이버예약 덕분에 에이드도 서비스로 나왔네요 가격은 비싸지...,59,분위기도 괜찮고 맛있어요 네이버예약 덕분에 에이드도 서비스로 나왔네요 가격은 비싸지...,59,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,백억하누 강남본점,2021-05-25,룸으로 이용했는데 화장실도 딸려있어서 너무 편했어요 삼합 재료들이 신선해서 생각보다...,265,룸으로 이용했는데 화장실도 딸려 있어서 너무 편했어요 삼합 재료들이 신선해서 생각보...,276,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7822 entries, 0 to 7821
Data columns (total 48 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   store_name    7822 non-null   object
 1   date          7822 non-null   object
 2   review        7822 non-null   object
 3   length        7822 non-null   int64 
 4   review_check  7822 non-null   object
 5   check_length  7822 non-null   int64 
 6   menu          7822 non-null   int64 
 7   mood          7822 non-null   int64 
 8   service       7822 non-null   int64 
 9   price         7822 non-null   int64 
 10  clean         7822 non-null   int64 
 11  room          7822 non-null   int64 
 12  parking       7822 non-null   int64 
 13  photo         7822 non-null   int64 
 14  sum           7822 non-null   int64 
 15  helpful       7822 non-null   int64 
 16  맛             7822 non-null   int64 
 17  맛집            7822 non-null   int64 
 18  메뉴_음식         7822 non-null   int64 
 19  친절_직원 

In [36]:
df.columns

Index(['store_name', 'date', 'review', 'length', 'review_check',
       'check_length', 'menu', 'mood', 'service', 'price', 'clean', 'room',
       'parking', 'photo', 'sum', 'helpful', '맛', '맛집', '메뉴_음식', '친절_직원', '때',
       '예약', '가격', '분위기', '추천', 'store', '딘타이펑', '백억하누', '화기애애', '라그릴리아',
       '강남진해장', '구스', '파파야리프', '고양이부엌', '시골야채된장', '을밀대', '담소', '청류벽', '기리야마',
       '시카고', '대려도', '신부자', '봉추찜닭', '리샨', 'Pavox', '삼거리', '느린마을', '봉피양'],
      dtype='object')

### (2) 가게 숫자 바꿔보기

#### 1) df
- 모든 가게 있는 그룹

In [40]:
df['store_name'].unique(), len(df['store_name'].unique())

(array(['백억하누 강남본점', '강남진해장', '딘타이펑 강남점', '삼거리포차', '화기애애 강남역점',
        '느린마을양조장 강남점', '시골야채된장', '봉피양제주 강남점', '라그릴리아 SPC스퀘어점', '청류벽',
        '대려도', '신부자식당', '리샨 삼성타운점', '구스아일랜드 브루하우스', '을밀대 강남점', '파파야리프',
        '고양이부엌', '오리지널시카고피자', 'Pavox', '우동명가기리야마본진', '담소사골순대 강남본점',
        '봉추찜닭 강남주공공이점'], dtype=object), 22)

#### 2) df2
- 리뷰 개수가 제일 많은(1000개 이상되는) 식당 제거 그룹
- 제거된 식당: 딘타이펑, 백억하누

In [41]:
df2=df.copy()
df2=df2[df2['store_name']!='딘타이펑 강남점']
df2=df2[df2['store_name']!='백억하누 강남본점']
df2=df2.reset_index(drop=True)
df2['store_name'].unique(), len(df2['store_name'].unique())

(array(['강남진해장', '삼거리포차', '화기애애 강남역점', '느린마을양조장 강남점', '시골야채된장',
        '봉피양제주 강남점', '라그릴리아 SPC스퀘어점', '청류벽', '대려도', '신부자식당', '리샨 삼성타운점',
        '구스아일랜드 브루하우스', '을밀대 강남점', '파파야리프', '고양이부엌', '오리지널시카고피자', 'Pavox',
        '우동명가기리야마본진', '담소사골순대 강남본점', '봉추찜닭 강남주공공이점'], dtype=object), 20)

#### 3) df3
- 리뷰 개수가 많은 상위 7개 식당 제거 그룹
- 제거된 식당: 딘타이펑, 백억하누, 느린마을양조장, 화기애애, 라그릴리아, 강남진해장, 구스아일랜드

In [42]:
df3=df.copy()
df3=df3[df3['store_name']!='딘타이펑 강남점']
df3=df3[df3['store_name']!='백억하누 강남본점']
df3=df3[df3['store_name']!='느린마을양조장 강남점']
df3=df3[df3['store_name']!='화기애애 강남역점']
df3=df3[df3['store_name']!='라그릴리아 SPC스퀘어점']
df3=df3[df3['store_name']!='강남진해장']
df3=df3[df3['store_name']!='구스아일랜드 브루하우스']
df3=df3.reset_index(drop=True)
df3['store_name'].unique(), len(df3['store_name'].unique())

(array(['삼거리포차', '시골야채된장', '봉피양제주 강남점', '청류벽', '대려도', '신부자식당', '리샨 삼성타운점',
        '을밀대 강남점', '파파야리프', '고양이부엌', '오리지널시카고피자', 'Pavox', '우동명가기리야마본진',
        '담소사골순대 강남본점', '봉추찜닭 강남주공공이점'], dtype=object), 15)

#### 4) df4
- 리뷰 개수가 중간층에 있는 식당 8개 제거 그룹
- 제거된 식당: 파파야리프, 고양리부엌, 시골야채된장, 을밀대, 담소사골순대, 청류벽, 우동명가기리야마본진, 신부자식당

In [39]:
df4=df.copy()
df4=df4[df4['store_name']!='파파야리프']
df4=df4[df4['store_name']!='고양이부엌']
df4=df4[df4['store_name']!='시골야채된장']
df4=df4[df4['store_name']!='을밀대 강남점']
df4=df4[df4['store_name']!='담소사골순대 강남본점']
df4=df4[df4['store_name']!='청류벽']
df4=df4[df4['store_name']!='우동명가기리야마본진']
df4=df4[df4['store_name']!='신부자식당']
df4=df4.reset_index(drop=True)
df4['store_name'].unique(), len(df4['store_name'].unique())

(array(['백억하누 강남본점', '강남진해장', '딘타이펑 강남점', '삼거리포차', '화기애애 강남역점',
        '느린마을양조장 강남점', '봉피양제주 강남점', '라그릴리아 SPC스퀘어점', '대려도', '리샨 삼성타운점',
        '구스아일랜드 브루하우스', '오리지널시카고피자', 'Pavox', '봉추찜닭 강남주공공이점'], dtype=object),
 14)

### (3) 토큰화 함수 만들기

#### 1) getNVM_lemma1
- NNG, NNP만 이용

In [44]:
def getNVM_lemma1(text):
    tokenizer = MeCab.Tagger()
    parsed = tokenizer.parse(text)
    #print(parsed)
    word_tag = [w for w in parsed.split("\n")] 
    pos = []
    tags = ['NNG', 'NNP']
   
    for word_ in word_tag[:-2]:
        word = word_.split('\t') #['아버지', 'NNG,*,F,아버지,*,*,*,*']
        tag = word[1].split(",")  #['EC', '*', 'F', '는다', '*', '*', '*', '*']
        # if('+' in tag[0]): #단어가 여러 형태소로 구성된 경우
        #     if ('VV' in tag[0] or 'VA' in tag[0] or 'VX' in tag[0]):
        #         t = tag[-1].split('/')[0]
        #         if t not in stop_words:
        #             pos.append(t)
        if tag[0] in tags:
          pos.append(word[0])
    return pos

### (4) Logistic Regression

#### 1) df + lemma1

##### tfidf, count Vector

In [46]:
tfidf = TfidfVectorizer(ngram_range=(1,2),tokenizer = getNVM_lemma1, preprocessor = None, lowercase = False)
cv = CountVectorizer(ngram_range=(1,2),tokenizer = getNVM_lemma1, preprocessor = None, lowercase = False)
tdm_tfidf = tfidf.fit_transform(df['review_check'].values.astype('U'))
tdm_cv= cv.fit_transform(df['review_check'].values.astype('U'))

##### x (feature)

In [56]:
df['store_name'].value_counts()

딘타이펑 강남점         1296
백억하누 강남본점        1111
느린마을양조장 강남점       624
화기애애 강남역점         504
라그릴리아 SPC스퀘어점     435
강남진해장             424
구스아일랜드 브루하우스      404
파파야리프             353
고양이부엌             331
시골야채된장            286
을밀대 강남점           283
담소사골순대 강남본점       278
청류벽               269
우동명가기리야마본진        268
신부자식당             247
오리지널시카고피자         183
대려도               157
봉추찜닭 강남주공공이점      131
리샨 삼성타운점          112
봉피양제주 강남점          71
Pavox              29
삼거리포차              26
Name: store_name, dtype: int64

In [57]:
x_tf = tdm_tfidf.toarray()
x_tf_len = []
for i in range(len(x_tf)):
    a = x_tf[i]
    a = np.append(a, np.log(df['check_length'][i]+1)) # 길이 컬럼 추가

    a = np.append(a, df['딘타이펑'][i])    
    a = np.append(a, df['백억하누'][i])
    a = np.append(a, df['느린마을'][i])   
    a = np.append(a, df['화기애애'][i])    
    a = np.append(a, df['라그릴리아'][i])    
    a = np.append(a, df['강남진해장'][i])    
    a = np.append(a, df['구스'][i])
    a = np.append(a, df['파파야리프'][i])   
    a = np.append(a, df['고양이부엌'][i])    
    a = np.append(a, df['시골야채된장'][i])    
    a = np.append(a, df['을밀대'][i])    
    a = np.append(a, df['담소'][i])
    a = np.append(a, df['청류벽'][i])    
    a = np.append(a, df['기리야마'][i])
    a = np.append(a, df['신부자'][i])    
    a = np.append(a, df['시카고'][i])    
    a = np.append(a, df['대려도'][i])    
    a = np.append(a, df['봉추찜닭'][i])    
    a = np.append(a, df['리샨'][i])
    a = np.append(a, df['봉피양'][i])       
    a = np.append(a, df['Pavox'][i])    
    a = np.append(a, df['삼거리'][i])  

    x_tf_len.append(a)

##### countvector 추가

In [58]:
x_cv = tdm_cv.toarray() # TDM (countvector) 추가
x_cv_len = []
for i in range(len(x_cv)):
    a = x_cv[i]
    a = np.append(a,np.log(df['check_length'][i]+1)) # 길이 컬럼 추가
 
    a = np.append(a, df['딘타이펑'][i])    
    a = np.append(a, df['백억하누'][i])
    a = np.append(a, df['느린마을'][i])   
    a = np.append(a, df['화기애애'][i])    
    a = np.append(a, df['라그릴리아'][i])    
    a = np.append(a, df['강남진해장'][i])    
    a = np.append(a, df['구스'][i])
    a = np.append(a, df['파파야리프'][i])   
    a = np.append(a, df['고양이부엌'][i])    
    a = np.append(a, df['시골야채된장'][i])    
    a = np.append(a, df['을밀대'][i])    
    a = np.append(a, df['담소'][i])
    a = np.append(a, df['청류벽'][i])    
    a = np.append(a, df['기리야마'][i])
    a = np.append(a, df['신부자'][i])    
    a = np.append(a, df['시카고'][i])    
    a = np.append(a, df['대려도'][i])    
    a = np.append(a, df['봉추찜닭'][i])    
    a = np.append(a, df['리샨'][i])
    a = np.append(a, df['봉피양'][i])       
    a = np.append(a, df['Pavox'][i])    
    a = np.append(a, df['삼거리'][i])  
 
    a = np.append(a, df['맛'][i])
    a = np.append(a, df['맛집'][i])
    a = np.append(a, df['메뉴_음식'][i])
    a = np.append(a, df['친절_직원'][i])
    a = np.append(a, df['때'][i])
    a = np.append(a, df['예약'][i])
    a = np.append(a, df['가격'][i])
    a = np.append(a, df['분위기'][i])
    a = np.append(a, df['추천'][i])  

    x_cv_len.append(a)

In [59]:
len(x_tf_len[0]), len(x_cv_len[0])

(20571, 20580)

##### y(target)

In [60]:
y = df['helpful']

##### 학습하기

In [62]:
x_train, x_test, y_train, y_test = train_test_split(x_cv_len, y, test_size=0.2, random_state=42)

In [70]:
clf_cv = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x_train,y_train)
pred = clf_cv.predict(x_test)
accuracy_score(y_test, pred)

0.8805111821086262

In [71]:
x2_train, x2_test, y2_train, y2_test = train_test_split(x_tf_len, y, test_size=0.2, random_state=42)

In [72]:
clf_tf = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x2_train,y2_train)
pred2 = clf_tf.predict(x2_test)
accuracy_score(y2_test, pred2)

0.8594249201277955

##### 결과
- countvector 추가한 그룹이 정확도 더 높음 (88% vs 85.9%)

#### 2) df2 + lemma1

##### tfidf, count Vector

In [73]:
tfidf = TfidfVectorizer(ngram_range=(1,2),tokenizer = getNVM_lemma1, preprocessor = None, lowercase = False)
cv = CountVectorizer(ngram_range=(1,2),tokenizer = getNVM_lemma1, preprocessor = None, lowercase = False)
tdm_tfidf = tfidf.fit_transform(df2['review_check'].values.astype('U'))
tdm_cv= cv.fit_transform(df2['review_check'].values.astype('U'))

##### x (feature)

In [74]:
df2['store_name'].value_counts()

느린마을양조장 강남점      624
화기애애 강남역점        504
라그릴리아 SPC스퀘어점    435
강남진해장            424
구스아일랜드 브루하우스     404
파파야리프            353
고양이부엌            331
시골야채된장           286
을밀대 강남점          283
담소사골순대 강남본점      278
청류벽              269
우동명가기리야마본진       268
신부자식당            247
오리지널시카고피자        183
대려도              157
봉추찜닭 강남주공공이점     131
리샨 삼성타운점         112
봉피양제주 강남점         71
Pavox             29
삼거리포차             26
Name: store_name, dtype: int64

In [75]:
x_tf = tdm_tfidf.toarray()
x_tf_len = []
for i in range(len(x_tf)):
    a = x_tf[i]
    a = np.append(a, np.log(df2['check_length'][i]+1)) # 길이 컬럼 추가

    a = np.append(a, df2['느린마을'][i])   
    a = np.append(a, df2['화기애애'][i])    
    a = np.append(a, df2['라그릴리아'][i])    
    a = np.append(a, df2['강남진해장'][i])    
    a = np.append(a, df2['구스'][i])
    a = np.append(a, df2['파파야리프'][i])   
    a = np.append(a, df2['고양이부엌'][i])    
    a = np.append(a, df2['시골야채된장'][i])    
    a = np.append(a, df2['을밀대'][i])    
    a = np.append(a, df2['담소'][i])
    a = np.append(a, df2['청류벽'][i])    
    a = np.append(a, df2['기리야마'][i])
    a = np.append(a, df2['신부자'][i])    
    a = np.append(a, df2['시카고'][i])    
    a = np.append(a, df2['대려도'][i])    
    a = np.append(a, df2['봉추찜닭'][i])    
    a = np.append(a, df2['리샨'][i])
    a = np.append(a, df2['봉피양'][i])       
    a = np.append(a, df2['Pavox'][i])    
    a = np.append(a, df2['삼거리'][i])  

    x_tf_len.append(a)

##### countvector 추가

In [76]:
x_cv = tdm_cv.toarray() # TDM (countvector) 추가
x_cv_len = []
for i in range(len(x_cv)):
    a = x_cv[i]
    a = np.append(a,np.log(df2['check_length'][i]+1)) # 길이 컬럼 추가
 
    a = np.append(a, df2['느린마을'][i])   
    a = np.append(a, df2['화기애애'][i])    
    a = np.append(a, df2['라그릴리아'][i])    
    a = np.append(a, df2['강남진해장'][i])    
    a = np.append(a, df2['구스'][i])
    a = np.append(a, df2['파파야리프'][i])   
    a = np.append(a, df2['고양이부엌'][i])    
    a = np.append(a, df2['시골야채된장'][i])    
    a = np.append(a, df2['을밀대'][i])    
    a = np.append(a, df2['담소'][i])
    a = np.append(a, df2['청류벽'][i])    
    a = np.append(a, df2['기리야마'][i])
    a = np.append(a, df2['신부자'][i])    
    a = np.append(a, df2['시카고'][i])    
    a = np.append(a, df2['대려도'][i])    
    a = np.append(a, df2['봉추찜닭'][i])    
    a = np.append(a, df2['리샨'][i])
    a = np.append(a, df2['봉피양'][i])       
    a = np.append(a, df2['Pavox'][i])    
    a = np.append(a, df2['삼거리'][i])  
 
    a = np.append(a, df2['맛'][i])
    a = np.append(a, df2['맛집'][i])
    a = np.append(a, df2['메뉴_음식'][i])
    a = np.append(a, df2['친절_직원'][i])
    a = np.append(a, df2['때'][i])
    a = np.append(a, df2['예약'][i])
    a = np.append(a, df2['가격'][i])
    a = np.append(a, df2['분위기'][i])
    a = np.append(a, df2['추천'][i])  

    x_cv_len.append(a)

In [78]:
len(x_tf_len[0]), len(x_cv_len[0])

(14248, 14257)

##### y(target)

In [77]:
y = df2['helpful']

##### 학습하기

In [79]:
x_train, x_test, y_train, y_test = train_test_split(x_cv_len, y, test_size=0.2, random_state=42)

In [80]:
clf_cv = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x_train,y_train)
pred = clf_cv.predict(x_test)
accuracy_score(y_test, pred)

0.8956602031394275

In [81]:
x2_train, x2_test, y2_train, y2_test = train_test_split(x_tf_len, y, test_size=0.2, random_state=42)

In [82]:
clf_tf = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x2_train,y2_train)
pred2 = clf_tf.predict(x2_test)
accuracy_score(y2_test, pred2)

0.863342566943675

##### 결과
- countvector 추가한 그룹이 정확도 더 높음 (89.6% vs 86.3%)

#### 3) df3 + lemma1

##### tfidf, count Vector

In [83]:
tfidf = TfidfVectorizer(ngram_range=(1,2),tokenizer = getNVM_lemma1, preprocessor = None, lowercase = False)
cv = CountVectorizer(ngram_range=(1,2),tokenizer = getNVM_lemma1, preprocessor = None, lowercase = False)
tdm_tfidf = tfidf.fit_transform(df3['review_check'].values.astype('U'))
tdm_cv= cv.fit_transform(df3['review_check'].values.astype('U'))

##### x (feature)

In [84]:
df3['store_name'].value_counts()

파파야리프           353
고양이부엌           331
시골야채된장          286
을밀대 강남점         283
담소사골순대 강남본점     278
청류벽             269
우동명가기리야마본진      268
신부자식당           247
오리지널시카고피자       183
대려도             157
봉추찜닭 강남주공공이점    131
리샨 삼성타운점        112
봉피양제주 강남점        71
Pavox            29
삼거리포차            26
Name: store_name, dtype: int64

In [86]:
x_tf = tdm_tfidf.toarray()
x_tf_len = []
for i in range(len(x_tf)):
    a = x_tf[i]
    a = np.append(a, np.log(df3['check_length'][i]+1)) # 길이 컬럼 추가

    a = np.append(a, df3['파파야리프'][i])   
    a = np.append(a, df3['고양이부엌'][i])    
    a = np.append(a, df3['시골야채된장'][i])    
    a = np.append(a, df3['을밀대'][i])    
    a = np.append(a, df3['담소'][i])
    a = np.append(a, df3['청류벽'][i])    
    a = np.append(a, df3['기리야마'][i])
    a = np.append(a, df3['신부자'][i])    
    a = np.append(a, df3['시카고'][i])    
    a = np.append(a, df3['대려도'][i])    
    a = np.append(a, df3['봉추찜닭'][i])    
    a = np.append(a, df3['리샨'][i])
    a = np.append(a, df3['봉피양'][i])       
    a = np.append(a, df3['Pavox'][i])    
    a = np.append(a, df3['삼거리'][i])  

    x_tf_len.append(a)

##### countvector 추가

In [87]:
x_cv = tdm_cv.toarray() # TDM (countvector) 추가
x_cv_len = []
for i in range(len(x_cv)):
    a = x_cv[i]
    a = np.append(a,np.log(df3['check_length'][i]+1)) # 길이 컬럼 추가

    a = np.append(a, df3['파파야리프'][i])   
    a = np.append(a, df3['고양이부엌'][i])    
    a = np.append(a, df3['시골야채된장'][i])    
    a = np.append(a, df3['을밀대'][i])    
    a = np.append(a, df3['담소'][i])
    a = np.append(a, df3['청류벽'][i])    
    a = np.append(a, df3['기리야마'][i])
    a = np.append(a, df3['신부자'][i])    
    a = np.append(a, df3['시카고'][i])    
    a = np.append(a, df3['대려도'][i])    
    a = np.append(a, df3['봉추찜닭'][i])    
    a = np.append(a, df3['리샨'][i])
    a = np.append(a, df3['봉피양'][i])       
    a = np.append(a, df3['Pavox'][i])    
    a = np.append(a, df3['삼거리'][i])  
 
    a = np.append(a, df3['맛'][i])
    a = np.append(a, df3['맛집'][i])
    a = np.append(a, df3['메뉴_음식'][i])
    a = np.append(a, df3['친절_직원'][i])
    a = np.append(a, df3['때'][i])
    a = np.append(a, df3['예약'][i])
    a = np.append(a, df3['가격'][i])
    a = np.append(a, df3['분위기'][i])
    a = np.append(a, df3['추천'][i])  

    x_cv_len.append(a)

In [88]:
len(x_tf_len[0]), len(x_cv_len[0])

(8256, 8265)

##### y(target)

In [89]:
y = df3['helpful']

##### 학습하기

In [90]:
x_train, x_test, y_train, y_test = train_test_split(x_cv_len, y, test_size=0.2, random_state=42)

In [91]:
clf_cv = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x_train,y_train)
pred = clf_cv.predict(x_test)
accuracy_score(y_test, pred)

0.9057851239669421

In [92]:
x2_train, x2_test, y2_train, y2_test = train_test_split(x_tf_len, y, test_size=0.2, random_state=42)

In [93]:
clf_tf = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x2_train,y2_train)
pred2 = clf_tf.predict(x2_test)
accuracy_score(y2_test, pred2)

0.9074380165289256

##### 결과
- tf 그룹이 정확도 더 높음 (90.6% vs 90.7%)

#### 4) df4 + lemma1

##### tfidf, count Vector

In [98]:
tfidf = TfidfVectorizer(ngram_range=(1,2),tokenizer = getNVM_lemma1, preprocessor = None, lowercase = False)
cv = CountVectorizer(ngram_range=(1,2),tokenizer = getNVM_lemma1, preprocessor = None, lowercase = False)
tdm_tfidf = tfidf.fit_transform(df4['review_check'].values.astype('U'))
tdm_cv= cv.fit_transform(df4['review_check'].values.astype('U'))

##### x (feature)

In [99]:
df4['store_name'].value_counts()

딘타이펑 강남점         1296
백억하누 강남본점        1111
느린마을양조장 강남점       624
화기애애 강남역점         504
라그릴리아 SPC스퀘어점     435
강남진해장             424
구스아일랜드 브루하우스      404
오리지널시카고피자         183
대려도               157
봉추찜닭 강남주공공이점      131
리샨 삼성타운점          112
봉피양제주 강남점          71
Pavox              29
삼거리포차              26
Name: store_name, dtype: int64

In [100]:
x_tf = tdm_tfidf.toarray()
x_tf_len = []
for i in range(len(x_tf)):
    a = x_tf[i]
    a = np.append(a, np.log(df4['check_length'][i]+1)) # 길이 컬럼 추가

    a = np.append(a, df4['딘타이펑'][i])    
    a = np.append(a, df4['백억하누'][i])
    a = np.append(a, df4['느린마을'][i])   
    a = np.append(a, df4['화기애애'][i])    
    a = np.append(a, df4['라그릴리아'][i])    
    a = np.append(a, df4['강남진해장'][i])    
    a = np.append(a, df4['구스'][i])   
    a = np.append(a, df4['시카고'][i])    
    a = np.append(a, df4['대려도'][i])    
    a = np.append(a, df4['봉추찜닭'][i])    
    a = np.append(a, df4['리샨'][i])
    a = np.append(a, df4['봉피양'][i])       
    a = np.append(a, df4['Pavox'][i])    
    a = np.append(a, df4['삼거리'][i])  

    x_tf_len.append(a)

##### countvector 추가

In [101]:
x_cv = tdm_cv.toarray() # TDM (countvector) 추가
x_cv_len = []
for i in range(len(x_cv)):
    a = x_cv[i]
    a = np.append(a,np.log(df4['check_length'][i]+1)) # 길이 컬럼 추가

    a = np.append(a, df4['딘타이펑'][i])    
    a = np.append(a, df4['백억하누'][i])
    a = np.append(a, df4['느린마을'][i])   
    a = np.append(a, df4['화기애애'][i])    
    a = np.append(a, df4['라그릴리아'][i])    
    a = np.append(a, df4['강남진해장'][i])    
    a = np.append(a, df4['구스'][i])   
    a = np.append(a, df4['시카고'][i])    
    a = np.append(a, df4['대려도'][i])    
    a = np.append(a, df4['봉추찜닭'][i])    
    a = np.append(a, df4['리샨'][i])
    a = np.append(a, df4['봉피양'][i])       
    a = np.append(a, df4['Pavox'][i])    
    a = np.append(a, df4['삼거리'][i])  

    a = np.append(a, df4['맛'][i])
    a = np.append(a, df4['맛집'][i])
    a = np.append(a, df4['메뉴_음식'][i])
    a = np.append(a, df4['친절_직원'][i])
    a = np.append(a, df4['때'][i])
    a = np.append(a, df4['예약'][i])
    a = np.append(a, df4['가격'][i])
    a = np.append(a, df4['분위기'][i])
    a = np.append(a, df4['추천'][i])  

    x_cv_len.append(a)

In [103]:
len(x_tf_len[0]), len(x_cv_len[0])

(15873, 15882)

##### y(target)

In [104]:
y = df4['helpful']

##### 학습하기

In [105]:
x_train, x_test, y_train, y_test = train_test_split(x_cv_len, y, test_size=0.2, random_state=42)

In [106]:
clf_cv = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x_train,y_train)
pred = clf_cv.predict(x_test)
accuracy_score(y_test, pred)

0.8557168784029038

In [107]:
x2_train, x2_test, y2_train, y2_test = train_test_split(x_tf_len, y, test_size=0.2, random_state=42)

In [108]:
clf_tf = LogisticRegression(random_state = 42, C=0.1, max_iter = 100).fit(x2_train,y2_train)
pred2 = clf_tf.predict(x2_test)
accuracy_score(y2_test, pred2)

0.8185117967332124

##### 결과
- countvector 추가한 그룹이 정확도 더 높음 (85.6% vs 81.9%)

#### lemma1(NNP, NNG) 정리 (cf vs tf)
- df: 88% vs 85.9%
- df2: 89.6% vs 86.3%
- df3: 90.6% vs 90.7%
- df4: 85.6% vs 81.9%
---------------------
- df: 모든 가게 있는 그룹
- df2: 리뷰 개수가 제일 많은(1000개 이상되는) 식당 제거 그룹
	- 제거된 식당: 딘타이펑, 백억하누
- df3: 리뷰 개수가 많은 상위 7개 식당 제거 그룹
  - 제거된 식당: 딘타이펑, 백억하누, 느린마을양조장, 화기애애, 라그릴리아, 강남진해장, 구스아일랜드
- df4: 리뷰 개수가 중간층에 있는 식당 8개 제거 그룹
  - 제거된 식당: 파파야리프, 고양리부엌, 시골야채된장, 을밀대, 담소사골순대, 청류벽, 우동명가기리야마본진, 신부자식당